In [52]:
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt

In [53]:
file_path = '../data/nba_points_2024_2025.xlsx'
df = pd.read_excel(file_path, usecols="A:T")

# Games that have been played and tracked
df = df[df['Game Count'] == 1]
# Remove whitespace from Home
df['Home'] = df['Home'].str.strip()
# Remove whitespace from Away
df['Away'] = df['Away'].str.strip()

In [54]:
df['Home Win'] = (df['Home Points'] > df['Away Points']).astype(int)
df['Away Win'] = (df['Away Points'] > df['Home Points']).astype(int)

In [55]:
print('GAMES TRACKED: ', round(df['Game Count'].sum()))
print('AVG. OPEN TOTAL: ', round(df['Open Total'].mean(), 2))
print('AVG. ACTUAL TOTAL: ', round(df['Actual Total'].mean(), 2))
print('OVERS: ', round(df['Over'].sum()/df['Game Count'].sum() * 100, 2), '%')
print('UNDERS: ', round(df['Under'].sum()/df['Game Count'].sum() * 100, 2), '%')

GAMES TRACKED:  890
AVG. OPEN TOTAL:  226.48
AVG. ACTUAL TOTAL:  226.91
OVERS:  52.47 %
UNDERS:  47.42 %


In [56]:
game_counts = df.groupby(['Crew Chief'])['Game Count'].sum().reset_index(name='games')
over_percs = round(df.groupby(['Crew Chief'])['Over'].sum()/df.groupby('Crew Chief')['Game Count'].sum() * 100, 2).reset_index(name='over_percentage')
over_by_avg = round(df.groupby(['Crew Chief'])['Over Amount'].mean(), 2).reset_index(name='over_by_avg')
over_by_std = round(df.groupby(['Crew Chief'])['Over Amount'].std(), 2).reset_index(name='over_by_std')
under_by_avg = round(df.groupby(['Crew Chief'])['Under Amount'].mean(), 2).reset_index(name='under_by_avg')
under_by_std = round(df.groupby(['Crew Chief'])['Under Amount'].std(), 2).reset_index(name='under_by_std')
under_percs = round(df.groupby(['Crew Chief'])['Under'].sum()/df.groupby('Crew Chief')['Game Count'].sum() * 100, 2).reset_index(name='under_percentage')
avg_totals = round(df.groupby(['Crew Chief'])['Actual Total'].mean(), 2).reset_index(name='mean_actual_total')
std_totals = round(df.groupby(['Crew Chief'])['Actual Total'].std(), 2).reset_index(name='std_actual_total')
avg_opening_total = round(df.groupby(['Crew Chief'])['Open Total'].mean(), 2).reset_index(name='mean_open_total')

groupby_results = [game_counts, over_percs, over_by_avg, over_by_std, under_by_avg, under_by_std, under_percs, avg_totals, std_totals, avg_opening_total]
chief_stats = reduce(lambda left, right: pd.merge(left, right, on='Crew Chief'), groupby_results)

In [57]:
chief_stats[(chief_stats.games >= 15) &
            ((chief_stats.over_percentage > 60) | (chief_stats.under_percentage > 60))  ][['Crew Chief', 'games', 'over_percentage', 'under_percentage']].sort_values(by='over_percentage')

,Crew Chief,games,over_percentage,under_percentage
9,Jacyn Goble,15.0,26.67,73.33
7,Ed Malloy,43.0,37.21,62.79
13,Josh Tiven,47.0,63.83,36.17
24,Scott Foster,40.0,65.00,35.00
28,Tyler Ford,49.0,67.35,32.65


In [58]:
# Get all unique team names from both Home and Away columns
teams = pd.concat([df['Home'], df['Away']]).unique()

# Initialize a list to store the stats for each team
stats = []

for team in teams:
    # Overall: games where the team is either home or away
    team_games = df[(df['Home'] == team) | (df['Away'] == team)]
    overall_over_pct = team_games['Over'].mean() * 100  # Proportion of games with Over
    
    # Home games only
    home_games = df[df['Home'] == team]
    home_over_pct = home_games['Over'].mean() * 100 if not home_games.empty else None
    
    # Away games only
    away_games = df[df['Away'] == team]
    away_over_pct = away_games['Over'].mean() * 100 if not away_games.empty else None
    
    stats.append({
        'Team': team,
        'Overall Over %': overall_over_pct,
        'Home Over %': home_over_pct,
        'Away Over %': away_over_pct
    })

# Convert the list of stats into a DataFrame
stats_df = pd.DataFrame(stats)

In [59]:
stats_df.sort_values('Overall Over %', ascending=False).head(10)

,Team,Overall Over %,Home Over %,Away Over %
23,MEM,69.491525,63.333333,75.862069
14,DEN,61.666667,68.965517,54.838710
17,CLE,60.344828,61.290323,59.259259
5,ATL,60.000000,67.857143,53.125000
9,UTA,59.322034,53.333333,65.517241
18,NYK,58.620690,58.064516,59.259259
3,PHI,57.627119,54.838710,60.714286
7,NOP,56.896552,56.666667,57.142857
28,OKC,55.000000,58.064516,51.724138
2,DET,55.000000,53.333333,56.666667
